# Algorithm Visualization and Performance Analysis

This notebook loads trained sign language recognition models, evaluates their performance on the validation dataset, 
and visualizes metrics such as accuracy, loss, and class-wise confusion matrices.

It also serves as the visualization and interpretability module for **Phase 1** of the Sign-to-Speech project.

In [ ]:
# Imports
import os
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from src.model import create_model
from src.dataset_loader import get_dataloaders  # to be defined in dataset_loader.py



In [ ]:
# Paths and parameters
CHECKPOINT_PATH = "models/checkpoints"  # change if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")
print("Available checkpoints:")
for f in os.listdir(CHECKPOINT_PATH):
    if f.endswith(".pt"):
        print(" -", f)


In [ ]:
# Get data loaders (assuming your dataset_loader.py defines this)
train_loader, val_loader, class_names = get_dataloaders(batch_size=32)

print(f"Classes ({len(class_names)}): {class_names}")


In [ ]:
# Select checkpoint (change filename to whichever version you want)
model_path = os.path.join(CHECKPOINT_PATH, "version_20251012_16_30_00.pt")

model = create_model(num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=DEVICE))
model.to(DEVICE)
model.eval()

print(f"✅ Loaded model: {model_path}")


In [ ]:
# Model Evaluation
correct, total = 0, 0
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
# Matrix Visualization
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


In [ ]:
# Prediction Visualization

def imshow(img, title=""):
    img = img / 2 + 0.5  # unnormalize if normalized
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis("off")

dataiter = iter(val_loader)
images, labels = next(dataiter)
images, labels = images.to(DEVICE), labels.to(DEVICE)

outputs = model(images)
_, preds = torch.max(outputs, 1)

# Display first 8 predictions
plt.figure(figsize=(15, 5))
for idx in range(8):
    plt.subplot(2, 4, idx+1)
    imshow(images[idx].cpu(), title=f"T: {class_names[labels[idx]]}\nP: {class_names[preds[idx]]}")
plt.tight_layout()
plt.show()


In [ ]:
# Training Performance Metrics

LOG_PATH = "models/logs/training_log.csv"  # update path if you save logs
if os.path.exists(LOG_PATH):
    df = pd.read_csv(LOG_PATH)
    plt.figure(figsize=(10,5))
    plt.plot(df["epoch"], df["train_loss"], label="Train Loss")
    plt.plot(df["epoch"], df["val_loss"], label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(df["epoch"], df["train_acc"], label="Train Acc")
    plt.plot(df["epoch"], df["val_acc"], label="Val Acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()
    plt.show()
else:
    print("No training log found — skip curve visualization.")


## Summary of Evaluation

- Loaded the trained model checkpoint
- Evaluated accuracy and generated classification report
- Visualized confusion matrix and sample predictions
- (Optional) Displayed training curves from logs

Next step: integrate real-time temporal correction visualization once the Phase 2 module is ready.
